In [ ]:
ImageSegmentation-SartorLab

In [ ]:
%%time
import os
HOME = os.getcwd()

In [ ]:
!pip install ultralytics==8.0.28

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image
import cv2
from ultralytics import YOLO
import numpy as np
import torch
import shutil
import csv
from pyzbar.pyzbar import decode
from pyzbar.pyzbar import ZBarSymbol
from plantcv import plantcv as pcv
import argparse
from pyzbar import pyzbar

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="toOloKd6frPuw92IVM3o")
project = rf.workspace("sartor-lab").project("duckweed-segmentation-2")
dataset = project.version(3).download("yolov8")

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolov8s-seg.pt data={dataset.location}/data.yaml epochs=10 imgsz=640

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train5/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train5/results.png', width=600)

In [ ]:
%cd {HOME}

!yolo task=segment mode=val model={HOME}/runs/segment/train5/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
# Running Yolo model on plant images
%%time
%cd {HOME}
import datetime

start_date = datetime.datetime(2023, 11, 9)
end_date = datetime.datetime(2023, 12, 4)

# image folder to run predict model on
drive_folder =  '/home/liftlab/mnt/gdrive/RaspberryPi/New_Images/ALPHA'
sub_folder = 'masks' # folder to save mask images in
sub_folder_2 = "barcode_images" # folder to save barcode images to
csv_folder = "csv" # folder to save csv file to
model = YOLO('/home/liftlab/ImageSegmentation/runs/segment/train5/weights/best.pt') # training model to predict with
results = model.predict(source= drive_folder, save=True, save_txt=False, stream=True, hide_labels=True, hide_conf=True, boxes=False)
csv_path = os.path.join(str(model.predictor.save_dir), csv_folder) # folder to save csv file to
subfolder_path = os.path.join(str(model.predictor.save_dir), sub_folder) ## creating subfolder for masks
subfolder_path_2 = os.path.join(str(model.predictor.save_dir), sub_folder_2) ## creating subfolder for barcode images
os.makedirs(subfolder_path, exist_ok=True)
os.makedirs(subfolder_path_2, exist_ok=True)
os.makedirs(csv_path, exist_ok=True)


for result in results: ## iterating through prediction results
    dir = str(model.predictor.save_dir)
    files = os.listdir(dir)
 
    for file in files: ## enusuring that only one segmented image stays in the folder at a time to ensure that 
        if "masks" in file or "barcode_images" in file or "csv" in file: ## skip over masks subfolder
            continue
        base, e = os.path.splitext(file) # split base name and extension
        # Construct the full path to the file
        file_path_r = os.path.join(dir, file)
        print(file)
        camera1 = file[8]
        date_str = file[10:20]
        time_str = file[21:29]
        date_str = date_str.replace('_', ':')
        timestamp_str = f"{date_str}_{time_str}".replace('_', ':')  # Combine date and time and replace underscores
        timestamp = datetime.datetime.strptime(timestamp_str, '%Y:%m:%d:%H:%M:%S')
        if (timestamp < start_date) or (timestamp > end_date):
            os.remove(file_path_r)
            continue
        
        if camera1 in ['2', '3', '6', '7']: #seperating plant images from barcode images
            os.remove(file_path_r)
        else:
            dest_path = os.path.join(subfolder_path_2, file) # move barcode images to 
            shutil.move(file_path_r, dest_path)

    if (timestamp < start_date) or (timestamp > end_date):
        continue
    
    if result.masks is None:
        continue

    if camera1 in ['2', '3', '6', '7']:
        masks = result.masks.data
        boxes = result.boxes.data
        clss = boxes[:,5]
        people_indices = torch.where(clss == 0)
        people_masks = masks[people_indices]
        people_mask = torch.any(people_masks, dim=0).int() * 255
        file_path = os.path.join(subfolder_path, f'{base}.jpg')
        cv2.imwrite(file_path, people_mask.cpu().numpy())

In [ ]:
%%time
upper_path = '/home/liftlab/ImageSegmentation'
full_path_1 = os.path.join(upper_path, subfolder_path)
full_path_2 = os.path.join(upper_path, subfolder_path_2)
full_path_csv = os.path.join(upper_path, csv_path)

In [ ]:
# This cell is used to iterate through all plant images and calculate their area and write the information to a csv
# file. If it is barcode image, this cell will detect the barcode and write the information to the file.
%%time
%cd {HOME}
filepath = os.path.join(full_path_csv, 'duckweed_data.csv') # file to write data to
with open(filepath, 'w', newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    # file header
    header = ['image path', 'timestamp', 'camera', 'sample', 'area', 'barcode'] #csv header
    csv_writer.writerow(header)
    print(len(os.listdir(subfolder_path)))
    if (len(os.listdir(subfolder_path)) > 0):
        print(20)
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg'): #skipping non jpg files
                img = cv2.imread(os.path.join(subfolder_path, filename))
                h, w, c = img.shape
                sub_w = w // 3  

                #splitting image into 3 section for each test tube depending on size of image
                if h > 480 :
                    sub_img1 = img[0:480, 70:sub_w+25, :]
                    sub_img2 = img[0:480, sub_w+25:2*sub_w-20, :]
                    sub_img3 = img[0:480, 2*sub_w-20:590, :]
                else:
                    sub_img1 = img[0:440, 70:sub_w+25, :]
                    sub_img2 = img[0:440, sub_w+25:2*sub_w-20, :]
                    sub_img3 = img[0:440, 2*sub_w-20:590, :]

                # Getting number of white pixels in each sub image to determine area of duckweed
                white_pix_sub1 = np.sum(sub_img1 == 255)
                white_pix_sub2 = np.sum(sub_img2 == 255)
                white_pix_sub3 = np.sum(sub_img3 == 255)
            
                full_path_file = os.path.join(drive_folder, filename) # original file path to write to csv file

                index = 8 # index of camera in filename
                camera = filename[index]
                timestamp = filename[10:29] 
                if camera in ['2', '3', '6', '7']: # Images with no barcode
                    filerow_1 = [ full_path_file, timestamp, camera, 0, white_pix_sub1, "N/A"]
                    filerow_2 = [ full_path_file, timestamp, camera, 1, white_pix_sub2, "N/A"]
                    filerow_3 = [ full_path_file, timestamp, camera, 2, white_pix_sub3, "N/A"]
                    csv_writer.writerow(filerow_1)
                    csv_writer.writerow(filerow_2)
                    csv_writer.writerow(filerow_3)

            else:
                print(f"skipping non jpg file")

    # iterate through and detect barcode images
    for filename in os.listdir(subfolder_path_2):
        
        if filename.endswith('.jpg'):
            # Construct the full path to the image file
            image_path = os.path.join(subfolder_path_2, filename)

            # Open the image using cv2
            img = cv2.imread(image_path)
            rotate_img = pcv.transform.rotate(img=img, rotation_deg=-90, crop=False)

            # Convert to grayscale (green-magenta channel)
            a = pcv.rgb2gray_lab(rgb_img=rotate_img, channel='a')

            # Create image mask
            img_binary = pcv.threshold.binary(gray_img=a, threshold=123, object_type='dark')

            # Fill in small objects (speckles)
            fill_image = pcv.fill(bin_img=img_binary, size=1000)

            # Loop to create a ROI and analyze the object within it for each of the 3 tubes
            img_copy = np.copy(rotate_img)

            for i in range(3):
                pcv.params.debug = "none"
            
                # Barcode detection
                if i == 2: # test tube 1
                        cropped_image = img[950:1500, 0:1000]
                if i == 1: # test tube 2
                        cropped_image = img[450:950, 0:1000] 
                if i == 0: # test tube 3
                        cropped_image = img[0:600, 0:1000]

                hsv = pcv.rgb2gray_hsv(cropped_image, channel='v')

                # Iterating thorugh image threshold and rotating image until barcode read
                for t in range(115,200):
                    binary = pcv.threshold.binary(hsv, threshold=t, object_type='light')
                    for r in range(-5,6):
                        rotated = pcv.transform.rotate(img=binary, rotation_deg=r, crop=False)
                        try:
                            barcodes = pyzbar.decode(rotated)
                        except Exception as e:
                            print(f"Error decoding barcode: {e}")
                            barcodes = []
                        if len(barcodes)==1: # Stop once barcode has been read
                                break
                    if len(barcodes)==1: # Stop once barcode has been read
                                break    
                
            
                for barcode in barcodes:
                        if len(barcodes) == 1: # Indicates that a barcode has been read
                            barcodeData = barcode.data.decode("utf-8")
                            pcv.outputs.add_observation(sample=str(i),method= "pyzbar", variable="Barcode",trait="Barcode",scale="String",datatype=str,value=barcodeData, label="Barcode")
                        else:
                            barcodeData = "N/A" # If no barcode read
                full_path_file = os.path.join(drive_folder, filename)
                camera = filename[8]
                timestamp = filename[10:29]
                if len(barcodes)==0: # if no barcode was detected
                    filerow_skip = [ full_path_file, timestamp, camera, i, "N/A", "Undetected"] 
                    csv_writer.writerow(filerow_skip)
                    continue;


                filerow = [ full_path_file, timestamp, camera, i, "N/A", barcodeData] # Write data for image to csv file
                csv_writer.writerow(filerow)

In [ ]:
# This cell is used to match images to their respective barcode
%%time
%cd {HOME}
import pandas as pd
from datetime import datetime, timedelta

num_area_rows = 10255 # Number of area rows to iterate through
num_barcode_rows = 10260 # Number of barcode rows to iterate through

df = pd.read_csv('/home/liftlab/ImageSegmentation/runs/segment/predict20/csv/duckweed_data.csv')

# takes in an area row, and iterates through all the barcode rows until it finds a match
def find_match(area_row, barcode_rows):
    for _, barcode_row in barcode_rows.iterrows():
        #print(area_row['timestamp'])
        area_timestamp = datetime.strptime(area_row['timestamp'], "%Y_%m_%d_%H:%M:%S")
        barcode_timestamp = datetime.strptime(barcode_row['timestamp'], "%Y_%m_%d_%H:%M:%S")

        # Check if the date is the same and the time difference is less than or equal to 2 seconds
        date_match = area_timestamp.date() == barcode_timestamp.date()
        
        time_diff = abs(area_timestamp - barcode_timestamp)
        time_match = time_diff <= timedelta(seconds=2)
        
        position_match = area_row['sample'] == barcode_row['sample'] # Check if they are in the same camera position
        
        cam = area_row['camera']
        cam_match = False
        # Camera 1 matches Camera 2
        if (cam == 2 and barcode_row['camera'] == 1):
            cam_match = True
        # Camera 3 matches Camera 4
        if (cam == 3 and barcode_row['camera'] == 4):
            cam_match = True 
        # Camera 5 matches Camera 6
        if (cam == 6 and barcode_row['camera'] == 5):
            cam_match = True 
        # Camera 7 matches Camera 8
        if (cam == 7 and barcode_row['camera'] == 8):
            cam_match = True 
        if date_match and time_match and position_match and cam_match: # If date, time, position, and camera match then the rows match and the barcode has been found
            print("YAY")
            return barcode_row['barcode']    

    return None

newPath = os.path.join('/home/liftlab/ImageSegmentation/runs/segment/predict20/csv/', 'merged_data.csv') # file to write data to

with open(newPath, 'w', newline='') as output_file:
    # Create a CSV writer
    csv_writer = csv.writer(output_file)

    head = ['image path', 'timestamp', 'camera', 'sample', 'area', 'barcode'] #csv header
    csv_writer.writerow(head)

    # Iterate through each area row to find a barcode match
    for area_index in range(num_area_rows):
        area_row = df.loc[area_index]
        barcode_rows = df.loc[num_area_rows: num_area_rows + num_barcode_rows - 1]
        matching_barcode = find_match(area_row, barcode_rows)
        
        # If match found, write barcode into column. Otherwise, write the row back as it was.
        if (matching_barcode != None):
            area_row['barcode'] = matching_barcode
        
        # Write the row to the output file
        csv_writer.writerow(area_row)